In [1]:
# https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html#

import os
from pathlib import Path

import pandas as pd
import numpy as np

import optuna
from optuna.trial import TrialState
from optuna.samplers import CmaEsSampler, RandomSampler, TPESampler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

In [2]:
REPO_PATH = '/home/ah19/runtime-monitoring'
DATASET = 'MNIST'
PREFIX = ''
FILENAME_POSTFIX = f"{DATASET}_{PREFIX}"
SEED = 42

In [3]:
DEVICE = torch.device("cuda:0")
EPOCHS = 5

In [4]:
import sys
sys.path.append(REPO_PATH + '/utilities')
sys.path.append(f'{REPO_PATH}/{DATASET}/trainingModels')

In [5]:
from utils import *
from plotFunctions import *
from MonitorUnifiedBDD import build_bdd
from pathManager import fetchPaths
from MNIST_Model import MNIST_Model

model_ = MNIST_Model

In [6]:
base = Path(REPO_PATH)
paths = fetchPaths(base, DATASET)

path = paths[DATASET.lower()]
path_dataset = paths['dataset']
path_trainingModels = paths['trainingModels']

path_trainingModels = path_trainingModels / 'optuna-last-hidden-layer'
path_trainingModels.mkdir(exist_ok=True)

path_trainingModels

PosixPath('/home/ah19/runtime-monitoring/MNIST/trainingModels/optuna-last-hidden-layer')

In [7]:
feature_names = get_labels(DATASET)

tf_train = T.Compose([
    T.ToTensor(),
    T.Normalize((0.1307), (0.3015))
])


tf_test = T.Compose([
    T.ToTensor(),
    T.Normalize((0.1307), (0.3015))
])

train_data = get_dataset(DATASET, path_dataset, train=True, transform=tf_train)
test_data = get_dataset(DATASET, path_dataset, train=False, transform=tf_test)

# train_split = int( len(train_data) * 0.2 )
# test_split = int( len(test_data) * 0.05 )

# train_data, _ = split_data(train_data, [train_split, len(train_data) - train_split], SEED)
# test_data, _ = split_data(test_data, [test_split, len(test_data) - test_split], SEED)

len(train_data), len(test_data)

(60000, 10000)

In [8]:
def get_loader():
    
    trainloader = get_dataLoader(train_data, 32, True)
    testloader = get_dataLoader(test_data, 32, False)
    
    return trainloader, testloader

In [9]:
def objective(trial):
    # Generate the model
    last_hidden_neurons_int = trial.suggest_int("last_hidden_neurons", 20, 120)
    
    model = model_(dropout=0.0, first_layer_norm=False,
                   last_hidden_neurons=last_hidden_neurons_int)

    model.to(DEVICE)
    
    # Generate the optimizers.
    optimizer = getattr(optim, 'SGD')(model.parameters(), lr=0.01)

    trainloader, testloader = get_loader()
    
    best_accuracy = 0
    
    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(trainloader):

            data, target = data.to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            
            # regularization
            l2_loss = model._l2_regularization(0.001)
            l1_loss = model._l1_regularization(0.001)
            
            # calc loss
            loss = F.cross_entropy(output, target) + l1_loss + l2_loss
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        losses = []
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(testloader):
                
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data)
                
                
                loss = F.cross_entropy(output, target)
                losses.append(loss.item())
                
                # Get the index of the max log-probability.
                correct += sum(target.to(DEVICE) == output.argmax(dim=1))
                
        accuracy = correct / len(testloader.dataset)
        
        if best_accuracy < accuracy:
            torch.save(model.state_dict(), path_trainingModels / 'chk.pth.tar')
            best_accuracy = accuracy
            

    model.load_state_dict(torch.load(path_trainingModels / 'chk.pth.tar'))

    export_last_hidden_layer(trainloader,model,DEVICE,
                             last_hidden_neurons_int, None, path_trainingModels, 'optuna', 'train')
    export_last_hidden_layer(testloader,model,DEVICE,
                             last_hidden_neurons_int, None, path_trainingModels, 'optuna', 'test')

    df = pd.read_csv(path_trainingModels / 'optuna_train.csv')
    df_test = pd.read_csv(path_trainingModels / 'optuna_test.csv')

    # split train data
    df_true = df[df['true'] == True].copy()
    df_true = df_true.drop('true', axis=1).reset_index(drop=True)


#         read data
    qnt_float = trial.suggest_float("quantile", 0, 0.95, step=0.01)
    thld = np.quantile(df_true.drop('y', axis=1), qnt_float, axis=0)

    res_info, res_scores = build_bdd((df.copy(), df_test.copy(), df_true.copy(), None,
                                      f'qt{round(qnt_float, 2)}', thld, 0, None))

    result = float(res_scores.loc[(res_scores['y'] == 'all')
                                  & (res_scores['stage'] == 'test')
                                  , 'outOfPatternMissClassified'].values[0])
    
    # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
        

    return result


In [10]:
total_time = int( 60 * 60 * 1.5 )
total_trials = 100

In [11]:
# params: L2, L1, bs, bn, drop, optim
#          3,  3,  4,  2,    6,     2 = 864

sampler = TPESampler(seed=SEED) # Default
# sampler = CmaEsSampler(seed=SEED)
# sampler = RandomSampler(seed=SEED)

study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=total_trials, timeout=total_time)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial
# trial = study.best_trials

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-04-09 16:06:10,269] A new study created in memory with name: no-name-38d7f1b3-6451-4c64-9155-0fc364bb124c


qt0.91 - eta: 0
Applying thresholds ... 
Adding patterns with no degree of freedom ... 
Done!
[==================================================]
Start Evaluating ...
Done!
[==================================================]
Start Evaluating ...
Done!
[==================================================]


[I 2023-04-09 16:15:19,071] Trial 0 finished with value: 0.031988873435326845 and parameters: {'last_hidden_neurons': 57, 'quantile': 0.91}. Best is trial 0 with value: 0.031988873435326845.


> Done! [ qt0.91 - eta: 0 ]


qt0.57 - eta: 0
Applying thresholds ... 
Adding patterns with no degree of freedom ... 


[W 2023-04-09 17:35:54,323] Trial 1 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/home/ah19/.local/lib/python3.6/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-9-1659335ade7c>", line 80, in objective
    f'qt{round(qnt_float, 2)}', thld, 0, None))
  File "/home/ah19/runtime-monitoring/utilities/MonitorUnifiedBDD.py", line 363, in build_bdd
    patterns.add_dataframe( df_true, eta)
  File "/home/ah19/runtime-monitoring/utilities/MonitorUnifiedBDD.py", line 138, in add_dataframe
    self.stats.loc[row, 'size_before_reorder_mb'] = round( asizeof.asizeof(self) * 1e-6, 3)
  File "/home/ah19/.local/lib/python3.6/site-packages/pympler/asizeof.py", line 2603, in asizeof
    s = _asizer.asizeof(*t)
  File "/home/ah19/.local/lib/python3.6/site-packages/pympler/asizeof.py", line 2032, in asizeof
    return sum(self._sizer(o, 0, 0, None) for o in objs)
  File "/home/

KeyboardInterrupt: 

In [ ]:
# optuna.visualization.plot_pareto_front(study, target_names=["loss", "accuracy"])

In [ ]:
# optuna.visualization.plot_param_importances(study, target=lambda t: t.values[0], target_name="accuracy")

In [ ]:
df_studys = study.trials_dataframe().sort_values('value', ascending=False)

In [ ]:
df_studys.to_csv(path_trainingModels / f'optunaResults_{type(sampler).__name__}.csv' ,index=False)

In [ ]:
df_studys = df_studys.set_index('number')

In [ ]:
df_studys.head()

In [ ]:
# params: L2, L1, bs, bn, drop, optim
#          3,  3,  4,  2,    6,     2 = 864

# sampler = TPESampler(seed=SEED) # Default
# sampler = CmaEsSampler(seed=SEED)
sampler = RandomSampler(seed=SEED)

study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=total_trials, timeout=total_time)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial
# trial = study.best_trials

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
# optuna.visualization.plot_pareto_front(study, target_names=["loss", "accuracy"])

In [ ]:
# optuna.visualization.plot_param_importances(
#     study, target=lambda t: t.values[0], target_name="accuracy"
# )

In [ ]:
df_studys = study.trials_dataframe().sort_values('value', ascending=False)

In [ ]:
df_studys.to_csv(path_trainingModels / f'optunaResults_{type(sampler).__name__}.csv' ,index=False)

In [ ]:
df_studys = df_studys.set_index('number')

In [ ]:
df_studys.head()

In [ ]:
# params: L2, L1, bs, bn, drop, optim
#          3,  3,  4,  2,    6,     2 = 864

# sampler = TPESampler(seed=SEED) # Default
sampler = CmaEsSampler(seed=SEED)
# sampler = RandomSampler(seed=SEED)

study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=total_trials, timeout=total_time)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial
# trial = study.best_trials

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
# optuna.visualization.plot_pareto_front(study, target_names=["loss", "accuracy"])

In [ ]:
# optuna.visualization.plot_param_importances(
#     study, target=lambda t: t.values[0], target_name="accuracy"
# )

In [ ]:
df_studys = study.trials_dataframe().sort_values('value', ascending=False)

In [ ]:
df_studys.to_csv(path_trainingModels / f'optunaResults_{type(sampler).__name__}.csv' ,index=False)

In [ ]:
df_studys = df_studys.set_index('number')

In [ ]:
df_studys.head()